In [1]:
import os
import pandas as pd 
import re
import json
import spacy
from collections import defaultdict
from itertools import product

2024-03-28 21:55:40.878652: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-28 21:55:40.916322: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-28 21:55:41.996464: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


##### Add Lookup List for Chemo
- before training my models, add some extra terms to expand the lexicons
- can't check fn here, here just compared with additional mention

In [2]:
# add indexs
breast_train_terms = ["docetaxel", "cyclophosphamide", \
                      "doxorubicin", \
                        'herceptin', "perjeta"]
breast_train_short_terms = ["chemo", "AC-T", "A/C"] # "Docetaxol"

# interferon need to add rule not add when it is a a/btest
# melanoma_train_terms = ["temozolomide", "tasisulam", "alfa-2b interferon", "vaccinia virus", \
#                         "alpha-2b interferon", "carboplatin", "Vaccinia  virus", "melphalan"]
melanoma_train_terms = ["temozolomide", "tasisulam", "vaccinia virus", \
                        "carboplatin", "Vaccinia  virus", "melphalan"]
melanoma_train_short_terms = ["chemo"]

# actually from test
ovarian_train_terms = ["taxol", "carboplatin", "cisplatin", "etoposide"]
ovarian_train_short_terms = ["chemo"]

def _get_seindexs_whole_note(string, spacy_better):
    doc = spacy_better(string)
    seindexs_whole_note = [] 
    for i, sent in enumerate(doc.sents):
        seindexs_whole_note.append((i, sent.start_char, sent.end_char))
    return seindexs_whole_note

def _find_nthind(tuple_list, start_ind):
    for tup in tuple_list:
        if tup[1] <= start_ind <= tup[2]:
            return tup[0]
    print("something wrong with this case")
    return None  # Return None if no match is found


# Lookup Add Chemos in Single Note
def _lookup_chemo(spacy_better, string, terms, short_terms, cancer, mode, patient_filename):
    """string: note with replacement \n"""
    sent_char_index_lst = _get_seindexs_whole_note(string, spacy_better) # (nth, start ind char of nth, end char ind of nth)
    matches = []
    if cancer == "breast": 
        pattern = r'\b' + re.escape("A.C") + r'\b'
        for match in re.finditer(pattern, string):
            start_index = match.start()
            end_index = match.end()
            matched_word = string[start_index:end_index]
            nth_sent = _find_nthind(sent_char_index_lst, start_index)
            matches.append((nth_sent, cancer, mode, patient_filename, matched_word, start_index, end_index))    # after mapping the vaccina virus first, then look for any virus left
    # if cancer == "melanoma": 
    #     pattern = r'\b' + re.escape("interferon") + r'\b'
    #     exclude_pattern1 = r'\b' + re.escape("alfa-2b interferon") + r'\b'
    #     exclude_pattern2 = r'\b' + re.escape("alpha-2b interferon") + r'\b'
    #     exclude_end1_lst, exclude_end2_lst = [], []
    #     for exclude_match1 in re.finditer(exclude_pattern1, string, re.IGNORECASE):
    #         exclude_end1_lst.append(exclude_match1.end())
    #     for exclude_match2 in re.finditer(exclude_pattern2, string, re.IGNORECASE):
    #         exclude_end2_lst.append(exclude_match2.end())
    #     for match in re.finditer(pattern, string, re.IGNORECASE):
    #         end_index = match.end()
    #         if (end_index not in exclude_end1_lst) and (end_index not in exclude_end2_lst): 
    #             start_index = match.start()
    #             end_index = match.end()
    #             matched_word = string[start_index:end_index]
    #             nth_sent = _find_nthind(sent_char_index_lst, start_index)
    #             matches.append((nth_sent, cancer, mode, patient_filename, matched_word, start_index, end_index))        
        pattern = r'\b' + re.escape("vaccinia") + r'\b'
        exclude_pattern1 = r'\b' + re.escape("vaccinia virus") + r'\b'
        exclude_pattern2 = r'\b' + re.escape("vaccinia  virus") + r'\b'
        exclude_start1_lst, exclude_start2_lst = [], []
        for exclude_match1 in re.finditer(exclude_pattern1, string, re.IGNORECASE):
            exclude_start1_lst.append(exclude_match1.start())
        for exclude_match2 in re.finditer(exclude_pattern2, string, re.IGNORECASE):
            exclude_start2_lst.append(exclude_match2.start())
        for match in re.finditer(pattern, string, re.IGNORECASE):
            start_index = match.start()
            if (start_index not in exclude_start1_lst) and (start_index not in exclude_start2_lst): 
                start_index = match.start()
                end_index = match.end()
                matched_word = string[start_index:end_index]
                nth_sent = _find_nthind(sent_char_index_lst, start_index)
                matches.append((nth_sent, cancer, mode, patient_filename, matched_word, start_index, end_index))            
    for term in terms:
        for match in re.finditer(re.escape(term), string, re.IGNORECASE):
            start_index = match.start()
            end_index = match.end()
            matched_word = string[start_index:end_index]
            nth_sent = _find_nthind(sent_char_index_lst, start_index)
            matches.append((nth_sent, cancer, mode, patient_filename, matched_word, start_index, end_index))
           # string[i] = string[:start_index] + '*' * (end_index - start_index) + string[end_index:]
    for term in short_terms:
        pattern = r'\b' + re.escape(term) + r'\b'
        for match in re.finditer(pattern, string, re.IGNORECASE):
            start_index = match.start()
            end_index = match.end()
            matched_word = string[start_index:end_index]
            nth_sent = _find_nthind(sent_char_index_lst, start_index)
            matches.append((nth_sent, cancer, mode, patient_filename, matched_word, start_index, end_index))
            # string[i] = string[:start_index] + '*' * (end_index - start_index) + string[end_index:]
    return matches



##### Add Lookup List for Timex

In [3]:
timex_terms = ["currently", "today"]
timex_short_terms = []

# Lookup Add Chemos in Single Note
def _lookup_timex(spacy_better, string, terms, short_terms, cancer, mode, patient_filename, dct):
    """string: note with replacement \n 
    dct: dct from this note"""
    if dct is None: 
        return []
    sent_char_index_lst = _get_seindexs_whole_note(string, spacy_better) # (nth, start ind char of nth, end char ind of nth)
    matches = []          
    for term in terms:
        for match in re.finditer(re.escape(term), string, re.IGNORECASE):
            start_index = match.start()
            end_index = match.end()
            matched_word = string[start_index:end_index]
            nth_sent = _find_nthind(sent_char_index_lst, start_index)
            matches.append((nth_sent, cancer, mode, patient_filename, matched_word, start_index, end_index, dct))
    return matches

##### Create New TSV File

In [4]:
# Create New Unsummarized File If Change
def new_tsv(UNSUMMARIZED_FILE, OUT_UNSUMMARIZED_FILE, more_df_tsv):
    unsum_df = pd.read_csv(UNSUMMARIZED_FILE, sep="\t")
    new_df = pd.concat([unsum_df, more_df_tsv], ignore_index=True, axis=0)
    new_df.to_csv(OUT_UNSUMMARIZED_FILE, sep="\t", index=False)

In [ ]:

# Apply to All File with Certain Cancer Type & Mode
def _lookup_cancer_modes(PATIENT_NOTE_DIR, terms, short_terms, timex_terms, timex_short_terms, cancer, mode):
    gold_dct_file = f"/users/the/NER_MTB/0_{cancer}_{mode}_gold_dct.json"
    with open(gold_dct_file, "r") as infile:
        gold_dct = json.load(infile)
    dct_lst, pi_lst, ct_lst, cai_lst, nt_lst, tai_lst, t_lst, nn_lst, ti_lst = [], [], [], [], [], [], [], [], []
    spacy_better = spacy.load("en_core_web_sm")
    for dir in os.listdir(PATIENT_NOTE_DIR):
        subfolder = os.path.join(PATIENT_NOTE_DIR, dir)
        if os.path.isdir(subfolder):
            for filename in os.listdir(subfolder):
                file = os.path.join(subfolder, filename)
                with open(file, "r") as infile:
                    lines = infile.readlines()
                # note with replacement \n
                line = lines[9:][0]
                if filename.split("_")[0] not in gold_dct: 
                    continue
                if "_".join(filename.split("_")[1:]).split(".txt")[0] in gold_dct[filename.split("_")[0]].keys(): 
                    chemo_lst = gold_dct[filename.split("_")[0]]["_".join(filename.split("_")[1:]).split(".txt")[0]]["pair_wise"]["chemo"]["ment"]
                    time_lst = gold_dct[filename.split("_")[0]]["_".join(filename.split("_")[1:]).split(".txt")[0]]["pair_wise"]["time"]["ment"]

                    dct_str = gold_dct[filename.split("_")[0]]["_".join(filename.split("_")[1:]).split(".txt")[0]]["doc_create_time"]
                    
                    add_timex_lst = _lookup_timex(spacy_better, line, timex_terms, timex_short_terms, cancer, mode, filename, dct_str["ment"])
                else:
                    chemo_lst = []
                    time_lst = []
                    add_timex_lst = []
                # note with replacement \n
                add_chemo_lst = _lookup_chemo(spacy_better, line, terms, short_terms, cancer, mode, filename)
                
                # check false positive
                add_chemo_text_lst = []
                add_timex_text_lst = []
                for row in add_chemo_lst:
                    add_chemo_text = row[4]
                    add_chemo_text_lst.append(add_chemo_text)
                for row in add_timex_lst:
                    add_timex_text = (row[4], row[-1])
                    add_timex_text_lst.append(add_timex_text)
                if len(chemo_lst) > 0:
                    # print(add_chemo_text_lst)
                    fp = set(add_chemo_text_lst)-set(chemo_lst)
                    if len(fp) > 0:
                        print(cancer, mode, filename)
                        print(f"false postive: {fp}")
                        print("\n")
                # if len(time_lst) > 0:
                #     add_timex_text_lst = [x[0] for x in add_timex_text_lst]
                #     fp = set(add_timex_text_lst)-set(time_lst)
                #     if len(fp) > 0:
                #         print(cancer, mode, filename)
                #         print(f"false postive: {fp}")
                #         print("\n")
                    # fn = set(time_lst) - set(add_timex_text_lst)
                    # if len(fn) > 0:
                    #     print(cancer, mode, filename)
                    #     print(f"still missing: {fn}")
                    #     print("\n")


                chemo_add_nthsent_lst, timex_add_nthsent_lst = [x[0] for x in add_chemo_lst], [x[0] for x in add_timex_lst]
                if len(chemo_add_nthsent_lst) == 0 or len(timex_add_nthsent_lst) == 0:
                    pass
                else: 
                    # can add more once have nth for all tsv file !!!!
                    for cind, chemo_nth in enumerate(chemo_add_nthsent_lst): 
                        for tind, timex_nth in enumerate(timex_add_nthsent_lst):
                            if abs(int(chemo_nth)-int(timex_nth)) == 0: 
                                yr, mn, da = add_timex_lst[tind][-1][:4], add_timex_lst[tind][-1][4:6], add_timex_lst[tind][-1][6:]
                                dct_lst.append(f"{yr}-{mn}-{da}")
                                pi_lst.append(filename.split("_")[0])
                                ct_lst.append(add_chemo_lst[cind][4])
                                cai_lst.append("100"+str(cind)+f"e@{filename.split('.txt')[0]}@system")
                                nt_lst.append(f"{yr}-{mn}-{da}")
                                tai_lst.append("100"+str(tind)+f"e@{filename.split('.txt')[0]}@system")
                                nn_lst.append(filename.split(".txt")[0])
                                t_lst.append("contains-1")
                                ti_lst.append("")
    df = pd.DataFrame()
    df = pd.DataFrame({"DCT":dct_lst, "patient_id":pi_lst, "chemo_text":ct_lst, \
                      "chemo_annotation_id": cai_lst, "normed_timex": nt_lst, \
                        "timex_annotation_id":tai_lst, "tlink": t_lst, \
                            "note_name":nn_lst, "tlink_inst":ti_lst})
    return df 

# Apply to All File
def add_lookup_main():
    # for cancer in ["breast", "ovarian", "melanoma"]:
    
    for cancer in ["melanoma"]: #, "melanoma"
        if cancer == "breast": 
            terms, short_terms = breast_train_terms, breast_train_short_terms
        elif cancer == "melanoma":
            terms, short_terms = melanoma_train_terms, melanoma_train_short_terms
        else:
            terms, short_terms = ovarian_train_terms, ovarian_train_short_terms
        for mode in ["test"]:
            UNSUMMARIZED_FILE = f"/users/the/NER_MTB/timelines/chemoTimelinesBaselineSystem/output/change_n2space_task2/unsummarized_{cancer}_{mode}_task2_output.tsv"
            OUT_UNSUMMARIZED_DIR = f"/users/the/NER_MTB/timelines/chemoTimelinesBaselineSystem/output/change_n2space_task2/add_lookup"
            os.makedirs(OUT_UNSUMMARIZED_DIR, exist_ok=True)
            OUT_UNSUMMARIZED_FILE = os.path.join(OUT_UNSUMMARIZED_DIR, F"unsummarized_{cancer}_{mode}_task2_output.tsv")

            PATIENT_NOTE_EXP_DIR = "/users/the/NER_MTB/timelines/chemoTimelinesBaselineSystem/input/change_n2space_task2"
            PATIENT_NOTE_DIR = os.path.join(PATIENT_NOTE_EXP_DIR, f"input_{cancer}_{mode}_task2", f"Patient_Notes", f"{cancer}", f"{mode}")
            print(f"{cancer}, {mode}")
            more_df_tsv = _lookup_cancer_modes(PATIENT_NOTE_DIR, terms, short_terms, timex_terms, timex_short_terms, cancer, mode)
            print(more_df_tsv)
            more_df_tsv.to_csv(f"./{cancer}_{mode}_task2_temp.csv")
            new_tsv(UNSUMMARIZED_FILE, OUT_UNSUMMARIZED_FILE, more_df_tsv)
            

add_lookup_main()